In [1]:
import pandas as pd
DATA_PATH = "../datasets/"

Load world population for computations:

In [2]:
world_pop = pd.read_csv(DATA_PATH + "world_population.csv")
# Rename column to match naming of other tables
world_pop = world_pop.rename(columns = {'CCA3': 'Code'}) 

In [3]:
print(f"Data available for {len(world_pop['Code'].unique())} countries.")
world_pop.head()

Data available for 234 countries.


,Rank,Code,Country/Territory,Capital,Continent,2022 Population,2020 Population,2015 Population,2010 Population,2000 Population,1990 Population,1980 Population,1970 Population,Area (km²),Density (per km²),Growth Rate,World Population Percentage
0,36,AFG,Afghanistan,Kabul,Asia,41128771,38972230,33753499,28189672,19542982,10694796,12486631,10752971,652230,63.0587,1.0257,0.52
1,138,ALB,Albania,Tirana,Europe,2842321,2866849,2882481,2913399,3182021,3295066,2941651,2324731,28748,98.8702,0.9957,0.04
2,34,DZA,Algeria,Algiers,Africa,44903225,43451666,39543154,35856344,30774621,25518074,18739378,13795915,2381741,18.8531,1.0164,0.56
3,213,ASM,American Samoa,Pago Pago,Oceania,44273,46189,51368,54849,58230,47818,32886,27075,199,222.4774,0.9831,0.00
4,203,AND,Andorra,Andorra la Vella,Europe,79824,77700,71746,71519,66097,53569,35611,19860,468,170.5641,1.0100,0.00


#### Temperatures

In [4]:
temperatures = pd.read_csv(DATA_PATH + "average-monthly-surface-temperature.csv")
# Rename columns based on their actual meaning explained on Kaggle
temperatures = temperatures.rename(columns = {'Average surface temperature': 'Monthly average surface temperature',
                                             'Average surface temperature.1': 'Yearly average surface temperature',
                                             'Day': 'Month', 'year': 'Year'})
temperatures['Month'] = pd.to_datetime(temperatures['Month']).dt.strftime("%m")

In [5]:
print(f"{temperatures.shape[0]} data points")
temperatures.head()

198900 data points


,Entity,Code,Year,Month,Monthly average surface temperature,Yearly average surface temperature
0,Afghanistan,AFG,1940,01,-2.032494,11.327695
1,Afghanistan,AFG,1940,02,-0.733503,11.327695
2,Afghanistan,AFG,1940,03,1.999134,11.327695
3,Afghanistan,AFG,1940,04,10.199754,11.327695
4,Afghanistan,AFG,1940,05,17.942135,11.327695


In [6]:
# Compute average monthly temperature for the last 10 years
temperatures_processed = temperatures[temperatures['Year'] > 2004]
temperatures_processed = temperatures_processed.groupby(['Code', 'Month'])["Monthly average surface temperature"].mean().reset_index()
temperatures_processed = temperatures_processed.rename(columns = {"Monthly average surface temperature": "Temperature"})
temperatures_processed.to_csv(DATA_PATH + 'processed/temperatures.csv')

In [7]:
print(temperatures_processed.shape)
temperatures_processed.head()

(2340, 3)


,Code,Month,Temperature
0,AFG,01,-1.049607
1,AFG,02,1.579556
2,AFG,03,7.712855
3,AFG,04,13.639239
4,AFG,05,19.318636


#### Inbound arrivals

In [67]:
inbound_arrivals = pd.read_csv(DATA_PATH + "23-international-tourist-trips-per-1000-people.csv")
print(inbound_arrivals.shape)
inbound_arrivals.head()

(4933, 4)


,Entity,Code,Year,Inbound arrivals (tourists) per 1000 people
0,Albania,ALB,2007,356.84418
1,Albania,ALB,2008,422.46985
2,Albania,ALB,2009,583.84890
3,Albania,ALB,2010,752.04175
4,Albania,ALB,2011,851.18560


In [76]:
# Compute average over last 5 available years, before COVID restrictions (2020)
inbound_arrivals_processed = inbound_arrivals[inbound_arrivals['Year'] < 2020].sort_values(
    by='Year', ascending=False).groupby('Code').head(5).groupby(
    'Code')['Inbound arrivals (tourists) per 1000 people'].mean().reset_index()
# Transform into absolute values to get country popularity
inbound_arrivals_processed = inbound_arrivals_processed.rename(
    columns = {"Inbound arrivals (tourists) per 1000 people": "Popularity"})
inbound_arrivals_processed = pd.merge(inbound_arrivals_processed, world_pop, how='inner', on='Code')
inbound_arrivals_processed['Popularity'] = inbound_arrivals_processed['Popularity'] * inbound_arrivals_processed['2022 Population'] / 1000
inbound_arrivals_processed = inbound_arrivals_processed[['Code', 'Popularity']].round()
inbound_arrivals_processed['Popularity'] = inbound_arrivals_processed['Popularity'].astype(int)
inbound_arrivals_processed.to_csv(DATA_PATH + "processed/popularity.csv")

In [79]:
inbound_arrivals_processed.sort_values(by='Popularity', ascending=False).head()

,Code,Popularity
60,FRA,87498049
189,USA,80166013
55,ESP,79783254
35,CHN,61819695
86,ITA,56588143


#### Trip budget

In [11]:
expenditures = pd.read_csv(DATA_PATH + "21-average-expenditures-of-international-tourists-domestically.csv")
print(expenditures.shape)
expenditures.head()

(1260, 4)


,Entity,Code,Year,Inbound Tourism Expenditure (adjusted for inflation and cost of living)
0,Australia,AUS,1995,12904206000
1,Australia,AUS,1996,13947016000
2,Australia,AUS,1997,14575643000
3,Australia,AUS,1998,14679026000
4,Australia,AUS,1999,16038053000


In [12]:
# Compute average over last 5 available years, before COVID restrictions (2020)
expenditures_processed = expenditures[expenditures['Year'] < 2020].sort_values(
    by='Year', ascending=False).groupby('Code').head(5).groupby(
    'Code')['Inbound Tourism Expenditure (adjusted for inflation and cost of living)'].mean().reset_index()
# Divide by absolute number of inbound arrivals to get average trip budget
expenditures_processed = expenditures_processed.rename(
    columns = {"Inbound Tourism Expenditure (adjusted for inflation and cost of living)": "Budget"})
expenditures_processed = pd.merge(expenditures_processed, absolute_arrivals, how='inner', on='Code')
expenditures_processed['Budget'] /= expenditures_processed['Popularity']
expenditures_processed = expenditures_processed[['Code', 'Budget']]
expenditures_processed.to_csv(DATA_PATH + "processed/budget.csv")

In [13]:
print(expenditures_processed.shape)
expenditures_processed.head()

(45, 2)


,Code,Budget
0,AUS,6106.631890
1,AUT,816.404918
2,BEL,1088.396149
3,BGR,1202.685054
4,BRA,1490.082091


#### Hotel occupancy

In [15]:
hotels = pd.read_csv(DATA_PATH + "15-foreign-guests-in-hotels-and-similar-establishments.csv")
print(hotels.shape)
hotels.head()

(2879, 4)


,Entity,Code,Year,Foreign guests in tourism accommodation (hotels and similar establishments)
0,Albania,ALB,1995,41000
1,Albania,ALB,1996,64000
2,Albania,ALB,1997,23000
3,Albania,ALB,1998,22000
4,Albania,ALB,1999,26000


In [82]:
# Compute average over last 5 available years, before COVID restrictions (2020)
hotels_processed = hotels[hotels['Year'] < 2020].sort_values(
    by='Year', ascending=False).groupby('Code').head(5).groupby(
    'Code')['Foreign guests in tourism accommodation (hotels and similar establishments)'].mean().reset_index()
hotels_processed = hotels_processed.rename(
    columns = {"Foreign guests in tourism accommodation (hotels and similar establishments)": "Hotel guests"})
hotels_processed["Hotel guests"] = hotels_processed["Hotel guests"].astype(int)
hotels_processed.to_csv(DATA_PATH + "processed/hotels.csv")

In [83]:
print(hotels_processed.shape)
hotels_processed.sort_values(by='Hotel guests', ascending=False).head()

(139, 2)


,Code,Hotel guests
23,CHN,97944800
124,THA,56006600
40,ESP,51890400
63,JPN,51055600
60,ITA,45213800


#### UNSECO World Heritage Sites

In [84]:
uwh_sites = pd.read_xml(DATA_PATH + "whc.unesco.org.xml")
print(uwh_sites.shape)
uwh_sites.head()

(1223, 21)


,category,criteria_txt,danger,date_inscribed,extension,http_url,id_number,image_url,iso_code,justification,...,location,longitude,region,revision,secondary_dates,short_description,site,states,transboundary,unique_number
0,Natural,(ix),None,2007,0,https://whc.unesco.org/en/list/1133,1133,https://whc.unesco.org/uploads/sites/site_1133...,"al,at,be,ba,bg,hr,cz,fr,de,it,pl,ro,sk,si,es,c...",None,...,None,22.183333,Europe and North America,0,"2011,2017,2021",<p>This transnational property includes 93 com...,Ancient and Primeval Beech Forests of the Carp...,"Albania,Austria,Belgium,Bosnia and Herzegovina...",1,2513
1,Mixed,(i)(iii)(iv)(vii),None,1979,1,https://whc.unesco.org/en/list/99,99,https://whc.unesco.org/uploads/sites/site_99.jpg,"al,mk",None,...,Ohrid (municipality),20.704167,Europe and North America,0,"2019,1980","<p>A superlative natural phenomenon, Lake Ohri...",Natural and Cultural Heritage of the Ohrid region,"Albania,North Macedonia",1,2313
2,Cultural,(i)(ii)(vi),None,2016,0,https://whc.unesco.org/en/list/1321,1321,https://whc.unesco.org/uploads/sites/site_1321...,"ar,be,fr,de,in,jp,ch",None,...,None,6.829336,Latin America and the Caribbean,0,None,"<p>Chosen from the work of Le Corbusier, the 1...","The Architectural Work of Le Corbusier, an Out...","Argentina,Belgium,France,Germany,India,Japan,S...",1,2085
3,Cultural,(ii)(iii)(iv)(vi),None,2014,0,https://whc.unesco.org/en/list/1459,1459,https://whc.unesco.org/uploads/sites/site_1459...,"ar,bo,cl,co,ec,pe",None,...,None,-69.591667,Latin America and the Caribbean,0,None,<p>This site is an extensive Inca communicatio...,"Qhapaq Ñan, Andean Road System","Argentina,Bolivia (Plurinational State of),Chi...",1,2003
4,Cultural,(iv),None,1983,1,https://whc.unesco.org/en/list/275,275,https://whc.unesco.org/uploads/sites/site_275.jpg,"ar,br",None,...,"State of Rio Grande do Sul, Brazil; Province o...",-54.265833,Latin America and the Caribbean,0,1984,<p>The ruins of São Miguel das Missões in Braz...,Jesuit Missions of the Guaranis: San Ignacio M...,"Argentina,Brazil",1,326


In [85]:
import pycountry

# Step 1: Copy and preprocess all rows
sites = uwh_sites.copy()

# Step 2: Split and standardize iso_code
sites["iso_code"] = sites["iso_code"].str.lower().str.split(",")

# Step 3: Explode list to individual codes
sites = sites.explode("iso_code")

# Step 4: Convert to 3-letter ISO codes using pycountry
def convert_to_iso3(code):
    try:
        return pycountry.countries.get(alpha_2=code.upper()).alpha_3
    except:
        return None

sites["Code"] = sites["iso_code"].apply(convert_to_iso3)

# Step 5: Drop rows with failed conversions
sites = sites.dropna(subset=["Code"])

# Step 6: Create two filtered DataFrames and count
natural_counts = (
    sites[sites["category"].isin(["Natural", "Mixed"])]["Code"]
    .value_counts()
    .sort_index()
    .reset_index()
    .rename(columns={"index": "Code", "count": "Natural sites"})
)
natural_counts.to_csv(DATA_PATH + "processed/natural_sites.csv")

cultural_counts = (
    sites[sites["category"].isin(["Cultural", "Mixed"])]["Code"]
    .value_counts()
    .sort_index()
    .reset_index()
    .rename(columns={"index": "Code", "count": "Cultural sites"})
)
cultural_counts.to_csv(DATA_PATH + "processed/cultural_sites.csv")

In [86]:
print(natural_counts.shape)
natural_counts.sort_values(by='Natural sites', ascending=False).head()

(114, 2)


,Code,Natural sites
19,CHN,19
2,AUS,16
106,USA,13
17,CAN,12
86,RUS,11


In [87]:
print(cultural_counts.shape)
cultural_counts.sort_values(by='Cultural sites', ascending=False).head()

(156, 2)


,Code,Cultural sites
66,ITA,54
34,DEU,51
41,ESP,46
46,FRA,46
26,CHN,44
